In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

%load_ext autoreload
%autoreload 2

from caltech256 import Caltech256

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/256_ObjectCategories'

In [3]:
caltech256_train = Caltech256(data_dir, data_transforms['train'], train=True)
caltech256_test = Caltech256(data_dir, data_transforms['train'], train=False)

In [4]:
vgg16 = models.vgg16_bn(pretrained=True)

In [5]:
vgg16.classifier.add_module('7', nn.Linear(in_features=1000, out_features=257))

In [6]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30)

vgg16 = nn.DataParallel(vgg16)
vgg16 = vgg16.cuda()

In [8]:
dataloader = DataLoader(caltech256_train, batch_size=4)
dataiter = iter(dataloader)
image, label = dataiter.next()
print(image.size())
print(label.size())

torch.Size([4, 3, 224, 224])
torch.Size([4])


In [9]:
def train_model(model, dataset, criterion, optimizer, scheduler, num_epochs, batch_size):
    start_time = time.time()
    model.train(True)
    dataset_size = dataset.__len__()
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        scheduler.step()
        running_loss = 0.
        batch_cnt = 0
        
        for data in dataloader:
            inputs, labels = data
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss
            
            batch_cnt += 1
            if batch_cnt % 150 == 0:
                print('Training completed [%d, %d]' % (epoch, batch_cnt))
            
            
        epoch_loss = running_loss / dataset_size
        print('%d epoch loss: %f' % (epoch, running_loss))
        
    model.train(False)
    time_elapsed = time.time() - start_time
    print('Training comple in %dm, %ds' % (time_elapsed//60, time_elapsed%60))
    return model

In [10]:
model_tf = train_model(vgg16, caltech256_train, criterion, optimizer, scheduler, num_epochs=5, batch_size=16)

Training completed [0, 500]
0 epoch loss: 2872.679443
Training completed [1, 500]
1 epoch loss: 2818.411865
Training completed [2, 500]
2 epoch loss: 2807.384766
Training completed [3, 500]
3 epoch loss: 2786.809082
Training completed [4, 500]
4 epoch loss: 2749.719482
Training comple in 9, 36


In [16]:
test_dataloader = DataLoader(caltech256_test, batch_size=16)
correct_cnt = 0
for data in test_dataloader:
    inputs, labels = data
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
    outputs = vgg16(inputs)
    _, preds = torch.max(outputs, 1)
    correct_cnt += torch.sum(pred.data == labels.data)
    
acc = correct_cnt / caltech256_test.__len__()
print('Test Set Accuracy: %f' % (acc*100))

Test Set Accuracy: 0.439453
